In [38]:
import json
import requests
import statistics as st
import pandas as pd
import numpy as np
import datetime as dt
import pyprind as bar
from dateutil import parser
from config import cur_lay_api

In [2]:
currencyDate = []
currencyId = []
currencyTitle = []
currencyConv = []
currencyVal = []
cur_con_pfx = "USD"
cur_conv = {"KPW":"North Korean Won",
            "KRW":"South Korean Won",
            "GBP":"British Pound Sterling",
            "EUR":"Euro",
            "HKD":"Hong Kong Dollar"}

In [3]:
currency_titles = pd.read_html("cur_table.html",encoding="latin-1")[0:]

In [4]:
titles = currency_titles[0]

In [5]:
start = dt.date(2018,5,15)
end = dt.date.today()
duration = (end - start).days

In [6]:
for cur in cur_conv.keys():
    progress = bar.ProgBar(duration, monitor=True, title="Retrieving historical currency data for {0}".format(cur))
    days = 0
    while days <= duration:
        date_it = (start+dt.timedelta(days=(days))).strftime("%Y-%m-%d")
        base_url = "https://apilayer.net/api/historical?access_key={0}&date={1}&source=USD&format=1".format(cur_lay_api,date_it)
        response = requests.get(base_url)
        data = response.json()
        currencyId.append(cur)
        currencyTitle.append(cur_conv[cur])
        currencyConv.append("USD")
        currencyDate.append(parser.parse(date_it))
        currencyVal.append(float(data["quotes"][cur_con_pfx+cur]))
        days+=1
    progress.update()

Retrieving historical currency data for KPW
Retrieving historical currency data for KRW
Retrieving historical currency data for GBP
Retrieving historical currency data for EUR
Retrieving historical currency data for HKD


In [7]:
col_dtypes = {"date":"datetime64[ns]",
              "currency":"object",
              "currencyName":"object",
              "toCurrency":"object",
              "rate":"float64"}

currency_df = pd.DataFrame(np.column_stack([currencyDate, currencyId, currencyTitle, currencyConv, currencyVal]),
                           columns=["date","currency","currencyName","toCurrency","rate"]).astype(col_dtypes)


In [8]:
currency_df.dtypes

date            datetime64[ns]
currency                object
currencyName            object
toCurrency              object
rate                   float64
dtype: object

In [10]:
currency_df["valueUSD"]=1/currency_df["rate"]

In [41]:
currencyRatePivot = pd.pivot_table(currency_df, values=["rate"], index=["currency","date"])

In [42]:
currencyUSDPivot = pd.pivot_table(currency_df, values=["valueUSD"], index=["currency","date"])

In [48]:
curRateDeviation = currencyRatePivot.std(level="currency")

In [49]:
curRateVariance = currencyRatePivot.var(level="currency")

In [50]:
curUSDDeviation = currencyUSDPivot.std(level="currency")

In [51]:
curUSDVariance = currencyUSDPivot.var(level="currency")